In [ ]:
!pip install benchmarx==0.0.11 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
# Imports
from benchmarx import Benchmark
from benchmarx.quadratic_problem import QuadraticProblem
import jax.numpy as jnp
from jax import random, grad
from jaxopt import BacktrackingLineSearch

In [ ]:
# Create Quadratic Problem

n = 5      # problem dimensionality

qp = QuadraticProblem(
    n=n,
    mineig=1,   # min eigenvalue
    maxeig=10   # max eigenvalue
)

# initial point
RNG = random.PRNGKey(1230)
RNG, _ = random.split(RNG)
x_init = random.normal(RNG, (n,))

In [ ]:
# Armijo line search
ls_armijo = BacktrackingLineSearch(
    fun=qp.f,
    maxiter=20,
    condition="armijo",
    decrease_factor=0.8
)
# Srong-Wolfe-condition line search
ls_strong_wolfe = BacktrackingLineSearch(
    fun=qp.f,
    maxiter=20,
    condition="strong-wolfe",
    decrease_factor=0.8
)

In [ ]:
benchmark = Benchmark(
    problem=qp,
    runs=2,
    methods= [
        {
            'GRADIENT_DESCENT_ls_armijo': {
                'x_init' : x_init,
                'tol': 0,
                'maxiter': 300,
                'stepsize' : 1e-1,
                'linesearch': ls_armijo,
                'acceleration': False,
                'label': 'GD_ls_armijo'
            },
            'GRADIENT_DESCENT_ls_strong_wolfe': {
                'x_init' : x_init,
                'tol': 0,
                'maxiter': 300,
                'stepsize' : 1e-1,
                'linesearch': ls_strong_wolfe,
                'acceleration': False,
                'label': 'GD_ls_strong_wolfe'
            }
        }
    ],
    metrics=[       # these data will be tracked as the experiment progresses
        "nit",
        "x",
        "f",
        "grad"
    ]
)

In [ ]:
result = benchmark.run()
result.plot(
    metrics=['f', 'x_norm', 'f_gap', 'x_gap', 'grad_norm']
)

In [ ]:
benchmark = Benchmark(
    problem=qp,
    runs=3,
    methods= [
        {
            'GRADIENT_DESCENT_armijo_step': {
                'x_init' : x_init,
                'tol': 1e-9,
                'maxiter': 300,
                'stepsize' : 0.1,
                'linesearch': 'backtracking',  # you can use default linesearch
                'condition': 'armijo',         # parameters
                'acceleration': False,
                'label': 'GD_armijo'
            },
            'GRADIENT_DESCENT_goldstein_step': {
                'x_init' : x_init,
                'tol': 1e-9,
                'maxiter': 300,
                'stepsize' : 0.1,
                'linesearch': 'backtracking', # you can use default linesearch
                'condition': 'goldstein',     # parameters
                'acceleration': False,
                'label': 'GD_goldstein'
            }
        }
    ],
    metrics=[
        "nit",
        "x",
        "f",
        "grad"
    ]
)

In [ ]:
result = benchmark.run()
result.plot(
        metrics=['f', 'x_norm', 'f_gap', 'x_gap', 'grad_norm']
    )